In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [11]:
train_data = pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv")
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [12]:
test_data = pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv")
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

In [13]:
y = np.log1p(train_data["SalePrice"])
X = train_data.drop(["Id", "SalePrice"], axis=1)

In [14]:
cat_features = X.select_dtypes(include=["object"]).columns.tolist()
print(cat_features)

X[cat_features] = X[cat_features].fillna("missing")
test_data[cat_features] = test_data[cat_features].fillna("missing")

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=1
)

train_pool = Pool(X_train, y_train, cat_features=cat_features)
valid_pool = Pool(X_valid, y_valid, cat_features=cat_features)

In [16]:
cat_model = CatBoostRegressor(
    iterations=2000,
    depth=8,
    learning_rate=0.05,
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=42,
    verbose=200,
    early_stopping_rounds=100
)

cat_model.fit(train_pool, eval_set=valid_pool)

0:	learn: 0.3814850	test: 0.4068460	best: 0.4068460 (0)	total: 114ms	remaining: 3m 47s
200:	learn: 0.0788792	test: 0.1296102	best: 0.1296102 (200)	total: 11.5s	remaining: 1m 42s
400:	learn: 0.0559147	test: 0.1254929	best: 0.1254623 (397)	total: 22.5s	remaining: 1m 29s
600:	learn: 0.0402465	test: 0.1235743	best: 0.1235743 (600)	total: 33.9s	remaining: 1m 18s
800:	learn: 0.0306073	test: 0.1229610	best: 0.1229419 (794)	total: 46.2s	remaining: 1m 9s
1000:	learn: 0.0237549	test: 0.1222427	best: 0.1222427 (1000)	total: 58.3s	remaining: 58.2s
1200:	learn: 0.0186659	test: 0.1220833	best: 0.1220304 (1161)	total: 1m 10s	remaining: 46.9s
1400:	learn: 0.0148781	test: 0.1219775	best: 0.1219430 (1320)	total: 1m 22s	remaining: 35.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.1219429592
bestIteration = 1320

Shrink model to first 1321 iterations.


In [17]:
preds_valid = cat_model.predict(X_valid)
rmse = np.sqrt(((preds_valid - y_valid) ** 2).mean())
print("Validation RMSE:", rmse)

Validation RMSE: 0.12194296865294502


In [18]:
feature_importances = cat_model.get_feature_importance(train_pool)
feat_imp = pd.DataFrame({
    "feature": X.columns,
    "importance": feature_importances
}).sort_values("importance", ascending=False)

feat_imp.head(10)

,feature,importance
16,OverallQual,15.020461
45,GrLivArea,13.186725
42,1stFlrSF,5.211618
37,TotalBsmtSF,4.952888
61,GarageArea,3.366483
3,LotArea,3.355323
33,BsmtFinSF1,2.984375
57,GarageType,2.800867
19,YearRemodAdd,2.739152
56,FireplaceQu,2.727247


In [20]:
preds_test = np.expm1(cat_model.predict(test_data.drop("Id", axis=1)))
submission = pd.DataFrame({"Id": test_data["Id"], "SalePrice": preds_test})
submission.to_csv("submission_catboost.csv", index=False)
submission.head()

,Id,SalePrice
0,1461,120268.354929
1,1462,164610.129267
2,1463,182775.206708
3,1464,195852.846534
4,1465,186821.494852
